<a href="https://colab.research.google.com/github/min-hee-Lee/python_workdemo/blob/master/p401_pandas_naver_movie_wordcloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###구글 코랩에서 한글 문제 해결
- 아래 코드 설치를 한다
- 코랩 런타임 -> 런타임 다시 시작을 수행한다.

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 1s (9,136 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

### 구글 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/python/data

Mounted at /content/drive
/content/drive/MyDrive/python/data


### import

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup


import pandas as pd


from tqdm.notebook import tqdm
import re

### Naver영화 - 평점순 영화별 제목, Review, 장르 Crawling

https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&date=20230219

읽어올 데이터의 진행률을 볼 때 tqdm을 사용한다.

In [ ]:
story = []
title = []
genre = []

for i in tqdm (range(1,2)) :  #1페이지 이상 2페이지 미만
  url = 'https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&date=20230219&page=' + str(i)
  html = urlopen(url)
  soup = BeautifulSoup(html, 'html.parser')
  #print(soup)
  #print(type(soup))
  #break

  titles = soup.find_all('div', class_='tit5')  # attrs={'class':'tit5'}
  href = []

  try :
      for each in titles :
        href.append(each.find('a')['href'])
      #print(href)

      for j in tqdm(range(len(href))):
        domain = 'https://movie.naver.com/'
        domain += href[j]
        #print(domain)
        html = urlopen(domain)
        soup = BeautifulSoup(html, 'html.parser')   #영화 선택해서 들어감
        genre_list = soup.select('dl.info_spec > dt.step1 + dd > p > span:nth-of-type(1) > a') #장르 가져오기
        #genre_text = []
        #print('genre_list:', len(genre_list))
        genre_text = []
        for k in range(len(genre_list)) : #장르 안에 a요소에 있는 글씨 가져오기
            #print(genre_list[k].get_text())
            genre_text.append(genre_list[k].get_text())
        genre.append(genre_text)

        #print(genre)

        title.append(soup.select_one('h3.h_movie > a').get_text())
        #print(title)

        story.append(soup.find('p', class_='con_tx').get_text())
        #print(story)

 
  except:
    pass


print(genre)
print(title)
print(story)


# 줄거리 정규화 처리
for i in range(len(story)) :
    story[i] = re.sub( '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》“”’{}]', '', story[ i ] )
    # select_one()와 select() 로 텍스트를 가져오면 <br>은 '\r'로, 공백(&nbsp;)은 '\xa0'로 가져온다.
    story[i] = re.sub( '\r\xa0', ' ', story[ i ] )
    
    print(story[i])

<ipython-input-23-c6ed3ec65d49>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook (range(1,2)) :  #1페이지 이상 2페이지 미만


  0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-23-c6ed3ec65d49>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for j in tqdm_notebook(range(len(href))):


  0%|          | 0/50 [00:00<?, ?it/s]

[['액션'], ['애니메이션', '코미디', '가족'], ['드라마'], ['드라마'], ['공연실황', '다큐멘터리'], ['다큐멘터리', '드라마'], ['다큐멘터리'], ['모험', '코미디', '드라마'], ['드라마'], ['드라마', '전쟁'], ['애니메이션', '모험'], ['애니메이션', '드라마', '판타지'], ['드라마', '액션'], ['SF', '모험', '액션', '전쟁'], ['액션', '드라마'], ['드라마', '멜로/로맨스'], ['드라마'], ['다큐멘터리'], ['애니메이션'], ['SF', '액션', '스릴러'], ['드라마'], ['멜로/로맨스', '드라마'], ['멜로/로맨스', '드라마'], ['전쟁', '액션', '드라마'], ['모험', '범죄', '가족', '코미디'], ['애니메이션', 'SF', '가족', '코미디', '멜로/로맨스', '모험'], ['공연실황'], ['SF', '코미디'], ['멜로/로맨스', '뮤지컬', '드라마'], ['드라마', '코미디'], ['드라마'], ['액션', '드라마'], ['드라마', '뮤지컬'], ['드라마'], ['드라마', '코미디'], ['범죄', '미스터리', '스릴러', '코미디', '드라마'], ['공연실황'], ['SF', '액션'], ['드라마'], ['드라마', '모험'], ['애니메이션', '판타지', '모험', '가족'], ['액션', '코미디'], ['애니메이션', '모험', '코미디', '가족', '판타지'], ['애니메이션'], ['다큐멘터리', '드라마'], ['드라마', '전쟁'], ['액션', '스릴러'], ['판타지', '모험', '액션', '전쟁'], ['드라마'], ['드라마', '가족']]
['탑건: 매버릭', '클라우스', '그린 북', '가버나움', '인생은 뷰티풀: 비타돌체', '밥정', '할머니의 먼 집', '베일리 어게인', '원더', '아일라', '디지몬 어드벤처 라스트 에볼루션 : 인연', '극장판 바이올렛 에버가든'

In [ ]:
print(len(title))
print(len(genre))
print(len(story))

50
50
50


### Pandas DataFrame 생성

In [ ]:
movie = pd.DataFrame(data={'title':title, 'story':story, 'genre':genre})
movie.shape

(50, 3)

In [ ]:
movie.head()

,title,story,genre
0,탑건: 매버릭,최고의 파일럿이자 전설적인 인물 매버릭톰 크루즈은 자신이 졸업한 훈련학교 교관으로 ...,[액션]
1,클라우스,편지 6천 통을 배달하라고요 소통은커녕 싸움만 일삼는 마을에서요 불가능해 보이는 임...,"[애니메이션, 코미디, 가족]"
2,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,[드라마]
3,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요 출생기록조차 없이 살아온 어쩌면...,[드라마]
4,인생은 뷰티풀: 비타돌체,언제나 화면 정 중앙에 비춰지며 스포트라이트를 한 몸에 받는 스타 트바로티 김호중 ...,"[공연실황, 다큐멘터리]"


In [ ]:
movie.tail()

,title,story,genre
45,쉰들러 리스트,1939년 독일에게 점령당한 폴란드의 한 도시 독일인 사업가이자 냉정한 기회주의자인...,"[드라마, 전쟁]"
46,헌터 킬러,미 국방부는 격침당한 잠수함의 행방을 찾기 위해 헌터 킬러를 극비리에 투입시키고 ...,"[액션, 스릴러]"
47,반지의 제왕: 왕의 귀환,모든 힘을 지배할 악의 군주 사우론의 절대반지가 깨어나고 악의 세력이 세상을 지배해...,"[판타지, 모험, 액션, 전쟁]"
48,죽은 시인의 사회,미국 입시 명문고 웰튼 아카데미 공부가 인생의 전부인 학생들이 아이비리그로 가기 위...,[드라마]
49,집으로...,도시에 사는 7살 개구쟁이 상우가 외할머니가 혼자 살고 계신 시골집에 머물게 된다 ...,"[드라마, 가족]"


In [ ]:
movie.iloc[0,1]

'최고의 파일럿이자 전설적인 인물 매버릭톰 크루즈은 자신이 졸업한 훈련학교 교관으로 발탁된다 그의 명성을 모르던 팀원들은 매버릭의 지시를 무시하지만 실전을 방불케 하는 상공 훈련에서 눈으로 봐도 믿기 힘든 전설적인 조종 실력에 모두가 압도된다  매버릭의 지휘아래 견고한 팀워크를 쌓아가던 팀원들에게 국경을 뛰어넘는 위험한 임무가 주어지자 매버릭은 자신이 가르친 동료들과 함께 마지막이 될 지 모를 하늘 위 비행에 나서는데'

### 영화 장르

In [ ]:
import numpy as np

genre_list = np.concatenate(movie['genre']).tolist()
print(genre_list)
print(len(genre_list))

genre_list = set(genre_list) #중복값 제거
print(genre_list)
print(len(genre_list))


['액션', '애니메이션', '코미디', '가족', '드라마', '드라마', '공연실황', '다큐멘터리', '다큐멘터리', '드라마', '다큐멘터리', '모험', '코미디', '드라마', '드라마', '드라마', '전쟁', '애니메이션', '모험', '애니메이션', '드라마', '판타지', '드라마', '액션', 'SF', '모험', '액션', '전쟁', '액션', '드라마', '드라마', '멜로/로맨스', '드라마', '다큐멘터리', '애니메이션', 'SF', '액션', '스릴러', '드라마', '멜로/로맨스', '드라마', '멜로/로맨스', '드라마', '전쟁', '액션', '드라마', '모험', '범죄', '가족', '코미디', '애니메이션', 'SF', '가족', '코미디', '멜로/로맨스', '모험', '공연실황', 'SF', '코미디', '멜로/로맨스', '뮤지컬', '드라마', '드라마', '코미디', '드라마', '액션', '드라마', '드라마', '뮤지컬', '드라마', '드라마', '코미디', '범죄', '미스터리', '스릴러', '코미디', '드라마', '공연실황', 'SF', '액션', '드라마', '드라마', '모험', '애니메이션', '판타지', '모험', '가족', '액션', '코미디', '애니메이션', '모험', '코미디', '가족', '판타지', '애니메이션', '다큐멘터리', '드라마', '드라마', '전쟁', '액션', '스릴러', '판타지', '모험', '액션', '전쟁', '드라마', '드라마', '가족']
108
{'애니메이션', '코미디', '모험', '판타지', '드라마', '다큐멘터리', '멜로/로맨스', '미스터리', '공연실황', 'SF', '가족', '전쟁', '뮤지컬', '스릴러', '범죄', '액션'}
16


### 전체 장르에 대한 코드 부여  
(genre unique 값(중복 제거한 값)을 쪼개서 번호 부여)

In [ ]:
genre_code_list_dict = dict((c, i) for i, c in enumerate(genre_list))
genre_code_list_dict

{'애니메이션': 0,
 '코미디': 1,
 '모험': 2,
 '판타지': 3,
 '드라마': 4,
 '다큐멘터리': 5,
 '멜로/로맨스': 6,
 '미스터리': 7,
 '공연실황': 8,
 'SF': 9,
 '가족': 10,
 '전쟁': 11,
 '뮤지컬': 12,
 '스릴러': 13,
 '범죄': 14,
 '액션': 15}

In [ ]:
# 전체 장르수
max_genre_code_length = len(genre_code_list_dict)
max_genre_code_length

16

### 각 영화별 장르 코드 변수 추가

In [ ]:
movie['genre_code'] = [[genre_code_list_dict[x] for x in element] for element in  movie['genre']]
movie

,title,story,genre,genre_code
0,탑건: 매버릭,최고의 파일럿이자 전설적인 인물 매버릭톰 크루즈은 자신이 졸업한 훈련학교 교관으로 ...,[액션],[15]
1,클라우스,편지 6천 통을 배달하라고요 소통은커녕 싸움만 일삼는 마을에서요 불가능해 보이는 임...,"[애니메이션, 코미디, 가족]","[0, 1, 10]"
2,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,[드라마],[4]
3,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요 출생기록조차 없이 살아온 어쩌면...,[드라마],[4]
4,인생은 뷰티풀: 비타돌체,언제나 화면 정 중앙에 비춰지며 스포트라이트를 한 몸에 받는 스타 트바로티 김호중 ...,"[공연실황, 다큐멘터리]","[8, 5]"
5,밥정,잔디 잡초 이끼 나뭇가지 자연을 재료 삼아 요리를 만드는 방랑식객 임지호 셰프 친...,"[다큐멘터리, 드라마]","[5, 4]"
6,할머니의 먼 집,아흔 셋 나의 사랑하는 할머니가 자살을 시도했다 취업준비를 하며 보내던 어느 날 ...,[다큐멘터리],[5]
7,베일리 어게인,귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다 하지만 눈을 떠보니 ...,"[모험, 코미디, 드라마]","[2, 1, 4]"
8,원더,누구보다 위트 있고 호기심 많은 매력 부자 어기제이콥 트렘블레이 하지만 남들과 다른...,[드라마],[4]
9,아일라,1950년 한국전쟁에 파병된 슐레이만은 칠흑 같은 어둠 속 홀로 남겨진 5살 소녀를...,"[드라마, 전쟁]","[4, 11]"


In [ ]:
movie.head()

,title,story,genre,genre_code
0,탑건: 매버릭,최고의 파일럿이자 전설적인 인물 매버릭톰 크루즈은 자신이 졸업한 훈련학교 교관으로 ...,[액션],[15]
1,클라우스,편지 6천 통을 배달하라고요 소통은커녕 싸움만 일삼는 마을에서요 불가능해 보이는 임...,"[애니메이션, 코미디, 가족]","[0, 1, 10]"
2,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,[드라마],[4]
3,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요 출생기록조차 없이 살아온 어쩌면...,[드라마],[4]
4,인생은 뷰티풀: 비타돌체,언제나 화면 정 중앙에 비춰지며 스포트라이트를 한 몸에 받는 스타 트바로티 김호중 ...,"[공연실황, 다큐멘터리]","[8, 5]"


In [ ]:
movie.tail()

,title,story,genre,genre_code
45,쉰들러 리스트,1939년 독일에게 점령당한 폴란드의 한 도시 독일인 사업가이자 냉정한 기회주의자인...,"[드라마, 전쟁]","[4, 11]"
46,헌터 킬러,미 국방부는 격침당한 잠수함의 행방을 찾기 위해 헌터 킬러를 극비리에 투입시키고 ...,"[액션, 스릴러]","[15, 13]"
47,반지의 제왕: 왕의 귀환,모든 힘을 지배할 악의 군주 사우론의 절대반지가 깨어나고 악의 세력이 세상을 지배해...,"[판타지, 모험, 액션, 전쟁]","[3, 2, 15, 11]"
48,죽은 시인의 사회,미국 입시 명문고 웰튼 아카데미 공부가 인생의 전부인 학생들이 아이비리그로 가기 위...,[드라마],[4]
49,집으로...,도시에 사는 7살 개구쟁이 상우가 외할머니가 혼자 살고 계신 시골집에 머물게 된다 ...,"[드라마, 가족]","[4, 10]"


### csv로 저장하기

In [ ]:
movie.to_csv('./movie_genre.csv')